<a href="https://colab.research.google.com/github/subbareddy248/LLaVA/blob/main/Video_LLaMA_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!apt -y install -qq aria2
!pip install gradio omegaconf salesforce-lavis bitsandbytes accelerate transformers==4.28.0
!git clone -b dev https://github.com/camenduru/Video-LLaMA-hf
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/Video-LLaMA-Series/resolve/main/ckpt/blip2_pretrained_flant5xxl.pth -d /content/Video-LLaMA-hf/ckpt -o blip2_pretrained_flant5xxl.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/Video-LLaMA-Series/resolve/main/ckpt/finetune-vicuna7b-v2.pth -d /content/Video-LLaMA-hf/ckpt -o finetune-vicuna7b-v2.pth
%cd /content/Video-LLaMA-hf
!python app.py

/content
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
E: Sub-process /usr/sbin/dpkg-preconfigure --apt || true received signal 2.
E: Failure running script /usr/sbin/dpkg-preconfigure --apt || true
debconf: apt-extracttemplates failed: No such file or directory
ERROR: Operation cancelled by user
Cloning into 'Video-LLaMA-hf'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 197 (delta 73), reused 169 (delta 58), pack-reused 0 (from 0)
Receiving objects: 100% (197/197), 476.54 KiB | 19.86 MiB/s, done.
Resolving deltas: 100% (73/73), done.
You can inspect what was checked out with 'git status'
and retry with 'git

In [1]:
from huggingface_hub import hf_hub_download
import torch
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration

# Load the model in half-precision
model = LlavaOnevisionForConditionalGeneration.from_pretrained("llava-hf/llava-onevision-qwen2-7b-ov-hf", torch_dtype=torch.float16, device_map="auto")
processor = AutoProcessor.from_pretrained("llava-hf/llava-onevision-qwen2-7b-ov-hf")

video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
conversation = [
    {

        "role": "user",
        "content": [
            {"type": "video", "path": video_path},
            {"type": "text", "text": "Why is this video funny?"},
            ],
    },
]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
inputs = processor.apply_chat_template(
    conversation,
    num_frames=8,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device, torch.float16)

out = model.generate(**inputs, output_hidden_states=True, return_dict_in_generate=True,max_new_tokens=60)
processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)
["user\n\nWhy is this video funny?\nassistant\nThe video appears to be humorous because it shows a young child, who is wearing glasses and holding a book, seemingly reading with a serious and focused expression. The child's glasses are a bit oversized for their face, which adds a comical touch, as it's a common trope to see children wearing"]

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


TypeError: argument 'ids': Can't extract `str` to `Vec`

In [8]:
out.keys()

odict_keys(['sequences', 'hidden_states', 'past_key_values'])

In [22]:
out['hidden_states'][30][0].shape

torch.Size([1, 1, 3584])

In [24]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values_videos'])

In [3]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 19.5 MB/s eta 0:00:00


In [25]:
!pip install qwen_vl_utils

In [3]:
from huggingface_hub import hf_hub_download
video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")


In [2]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model_path = "OpenGVLab/VideoChat-R1_7B"
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_path, device_map="auto",torch_dtype=torch.float16,
)

# default processer
processor = AutoProcessor.from_pretrained(model_path)

#video_path = "your_video.mp4"
question = "Where is the final cup containing the object?"



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


NameError: name 'video_path' is not defined

In [4]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": video_path,
                "max_pixels": 460800,
                "nframes": 32
            },
            {"type": "text", "text": f"""{question}
            Provide your final answer within the <answer> </answer> tags.
             """},
        ],
    }
]



#In Qwen 2.5 VL, frame rate information is also input into the model to align with absolute time.
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs, video_kwargs = process_vision_info(messages, return_video_kwargs=True)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
    **video_kwargs,
)
inputs = inputs.to("cuda")

# Inference
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


qwen-vl-utils using torchvision to read video.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.37 GiB. GPU 0 has a total capacity of 14.74 GiB of which 442.12 MiB is free. Process 108051 has 14.31 GiB memory in use. Of the allocated memory 13.07 GiB is allocated by PyTorch, and 1.10 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [2]:
output_text

NameError: name 'output_text' is not defined

In [14]:
video_path

'/root/.cache/huggingface/hub/datasets--raushan-testing-hf--videos-test/snapshots/655c9ae43a5a61f91aa3662d115c677b7d760d1e/sample_demo_1.mp4'